# Hyperparameter Tuning Pipeline

This notebook was solely run in a Google Colab T4 Python 3 environment. It took roughly 15 minutes to optimize the hyperparameters excluding the installation of dependencies.

In order to obtain the parameters used in the thesis, please run this notebook in Google Colab with a T4 Python 3 runtime.

# Install Dependencies

In [1]:
!pip install optuna==4.3.0
!pip install pandas==2.2.2
!pip install torch==2.6.0
!pip install numpy==2.0.2
!pip install matplotlib
!pip install scipy==1.15.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlin

# Import Libraries & Load Data

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from datetime import timedelta
import datetime as dt
import random
import optuna
from optuna.trial import TrialState
import os

# Read the CSV and convert date to datetime
df = pd.read_csv('./dji_etc_cc_ohlc_data.csv').drop(columns='Unnamed: 0')
df['date'] = pd.to_datetime(df['date'])
risk_free_rate = pd.read_csv('./risk_free_rate.csv')
risk_free_rate.columns = ['date', 'yield']
risk_free_rate.set_index('date', inplace=True)
risk_free_rate = risk_free_rate['yield']/100
risk_free_rate.index = pd.to_datetime(risk_free_rate.index)

# Filter to DJIA tickers drop DOW Inc. due to insufficient data
dji_ticker = ['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS',
              'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM',
              'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT',
              'IGLN.L', 'ISLN.L', 'IPLT.L', 'IPDM.L']

def set_seed(seed):
    """Set all random seeds for reproducibility."""
    random.seed(seed)          # Python
    np.random.seed(seed)       # NumPy
    torch.manual_seed(seed)    # PyTorch CPU
    torch.cuda.manual_seed(seed)       # PyTorch GPU (current CUDA device)
    torch.cuda.manual_seed_all(seed)   # PyTorch GPU (all CUDA devices)
    torch.backends.cudnn.deterministic = True  # CuDNN determinism
    torch.backends.cudnn.benchmark = False     # Disable CuDNN benchmarking

set_seed(7)

In [3]:
# Check Versions
print(f'Pandas Version: {pd.__version__}')
print(f'Numpy Version: {np.__version__}')
print(f'PyTorch Version: {torch.__version__}')
print(f'Optuna Version: {optuna.__version__}')


Pandas Version: 2.2.2
Numpy Version: 2.0.2
PyTorch Version: 2.6.0+cu124
Optuna Version: 4.3.0


# Declaration of Functions

In [4]:
# -----------------------------------------
# Data Preparation and Feature Engineering
# -----------------------------------------

def prepare_data(long_df,):
    """
    Pivot the long dataframe into a wide format with 'close' prices and calculate
    technical indicators (SMA and RSI) for each asset.
    """
    # Pivot to wide format for close prices only
    # Filter to DJIA tickers
    dji_ticker = ['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS',
                'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM',
                'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT',
                'IGLN.L', 'ISLN.L', 'IPLT.L', 'IPDM.L']

    df_filtered = long_df[long_df['tic'].isin(dji_ticker)].reset_index(drop=True)
    wide_df = df_filtered.pivot(index='date', columns='tic', values='close')
    wide_df.columns = [f'close_{col}' for col in dji_ticker]
    wide_df = wide_df.sort_index()
    wide_df = wide_df.ffill().bfill()

    # Calculate technical indicators for each ticker
    for tic in dji_ticker:
        close_col = f'close_{tic}'
        # Simple Moving Average (SMA)
        wide_df[f'sma_{tic}'] = wide_df[close_col].rolling(20).mean()

        # Relative Strength Index (RSI)
        delta = wide_df[close_col].diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(14).mean()
        avg_loss = loss.rolling(14).mean()
        rs = avg_gain / avg_loss
        wide_df[f'rsi_{tic}'] = 100 - (100 / (1 + rs))

    return wide_df.bfill()  # Backfill missing values

def extract_features(wide_df, rebalance_date, lookback=252):
    """
    Extract features from a lookback window ending at a given rebalance_date.
    Features include:
      - The latest SMA and RSI values (per ticker)
      - Flattened covariance matrices for the past 1-year of:
            close prices, SMAs, and RSIs.
    """
    lookback_data = wide_df.loc[:rebalance_date].iloc[-lookback:]
    # Get the list of tickers from the columns, assuming they are named like "close_<tic>"
    tics = [col.split('_')[1] for col in wide_df.columns if col.startswith('close_')]
    features = []
    # Covariance matrices for close, sma, and rsi data
    close_data = lookback_data[[f'close_{tic}' for tic in tics]].values
    sma_data = lookback_data[[f'sma_{tic}' for tic in tics]].values
    rsi_data = lookback_data[[f'rsi_{tic}' for tic in tics]].values

    for i in [close_data, sma_data, rsi_data]:
        features.append(i)

    return np.array(features)

# -------------------------
# Neural Network Definition
# -------------------------
class TimeSeriesNetwork(nn.Module):
    def __init__(self, num_tickers=33):
        super(TimeSeriesNetwork, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=16, kernel_size=5, padding=2)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveMaxPool1d(output_size=1)

        self.fc1 = nn.Linear(in_features=32, out_features=16)
        self.fc2 = nn.Linear(in_features=16, out_features=1)  # Score per ticker

        self.num_tickers = num_tickers

    def forward(self, x):
        # x: (batch, 3, 252, num_tickers)
        batch_size = x.size(0)
        x = x.permute(0, 3, 1, 2)  # (batch, num_tickers, 3, 252)
        x = x.reshape(batch_size * self.num_tickers, 3, 252)

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x).squeeze(-1)  # (batch*num_tickers, 32)

        x = F.relu(self.fc1(x))
        scores = self.fc2(x).view(batch_size, self.num_tickers)  # (batch, num_tickers)

        weights = F.softmax(scores, dim=1).unsqueeze(1)  # (batch, 1, num_tickers)
        return weights

#--------------------------
# Loss Function
#--------------------------

def sharpe_ratio_loss(returns, weights, rfr=0.024):
    """
    Compute differentiable Sharpe ratio loss.
    - returns: (batch, time, assets)
    - weights: (batch, 1, assets)
    """
    # Weighted portfolio returns per timestep
    port_returns = (returns * weights).sum(dim=2)  # (batch, time)
    daily_rfr = (1 + rfr) ** (1 / 252) - 1
    excess_returns = port_returns - daily_rfr

    mean_ret = excess_returns.mean(dim=1)
    std_ret = excess_returns.std(dim=1) + 1e-6
    sharpe = mean_ret / std_ret

    return -sharpe.mean()  # negative for minimization

def loss_function(returns, weights, rfr=0.024, reg_lambda=0.01, div_lambda=0.01):
    """
    Sharpe loss + L2 regularization + diversification penalty
    """
    sr_loss = sharpe_ratio_loss(returns, weights, rfr)

    # Regularization term
    reg = reg_lambda * (weights ** 2).sum()

    # Entropy-based diversification reward
    entropy = - (weights * torch.log(weights + 1e-6)).sum(dim=2).mean()
    div_penalty = -div_lambda * entropy

    return sr_loss + reg + div_penalty

# -------------------------
# Training Loop
# -------------------------

def train_model(model, train_features, train_targets, num_epochs=100, lr=0.001, rfr=0.024, reg_lambda=0.01, div_lambda=0.01):
    """
    Train the model on a growing dataset of past lookback windows.
    """
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    dataset = TensorDataset(train_features, train_targets)
    loader = DataLoader(dataset, batch_size=16, shuffle=True)

    for epoch in range(num_epochs):
        epoch_loss = []
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            weights = model(X_batch).squeeze(1)  # shape (batch, assets)
            loss = loss_function(y_batch, weights.unsqueeze(1), rfr=rfr, reg_lambda=reg_lambda, div_lambda=div_lambda)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss.append(loss.item())

        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}: Avg Loss = {-np.mean(epoch_loss):.4f}")

    return model


In [5]:
# ---------------------------
# Prepare Base Data and Rebalance Dates
# ---------------------------
long_df = df  # original dataframe
wide_df = prepare_data(long_df)
tics = dji_ticker

# Generate quarter ends within the date range of your DataFrame
quarter_ends = pd.date_range(start=wide_df.index.min(), end=wide_df.index.max(), freq='QE')
indices = wide_df.index.get_indexer(quarter_ends, method='pad')
valid_indices = [i for i in indices if i != -1]
adjusted_dates = wide_df.index[valid_indices]
train_end_dates = adjusted_dates[3:]
train_start_dates = adjusted_dates[:-3]

# Optimization

In [6]:

# -------------------------
# Optuna Objective Function for Hyperparameter-Tuning
# -------------------------
def objective(trial, model, train_features, train_targets, rfr):
    reg_lambda = trial.suggest_float("reg_lambda", 1e-5, 1e-1, log=True) # Suggests a number on interval [1e-1,1e-5]
    div_lambda = trial.suggest_float("div_lambda", 1e-5, 1e-1, log=True) # Suggests a number on interval [1e-1,1e-5]
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True) # Suggests a number on interval [1e-2,1e-5]

    # Same training logic for TimeSeriesNetwork just different parameters for learning-rate, diversification and regularization lambda
    temp_model = TimeSeriesNetwork(num_tickers=model.num_tickers).to(device)
    temp_model.load_state_dict(model.state_dict())
    optimizer = torch.optim.Adam(temp_model.parameters(), lr=lr)

    dataset = TensorDataset(train_features, train_targets)
    loader = DataLoader(dataset, batch_size=16, shuffle=True)

    temp_model.train()
    # Using fewer epochs for faster runtime
    for epoch in range(20):
        total_loss = 0
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            weights = temp_model(X_batch)
            loss = loss_function(y_batch, weights, rfr=rfr, reg_lambda=reg_lambda, div_lambda=div_lambda)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(loader)
        trial.report(avg_loss, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return avg_loss

# -------------------------
# Main Backtest Loop with Tuning
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Initialize data structures
optimal_params = {
    'reg_lambda': [],
    'div_lambda': [],
    'lr': []
}

model = TimeSeriesNetwork(num_tickers=len(tics)).to(device)
train_features_list = []
train_targets_list = []
allocation_weights = []
portfolio_returns = []
backtest_dates = []

# Backtest loop
for i in range(len(train_end_dates)-1):
    train_end = train_end_dates[i]
    test_start = train_end_dates[i]
    test_end = train_end_dates[i+1]

    # Prepare data
    feat = extract_features(wide_df, train_end)
    rets = wide_df[[f'close_{tic}' for tic in tics]].pct_change().fillna(0)
    rets = rets.loc[:train_end].iloc[-252:].values
    rfr = risk_free_rate.loc[:train_end].iloc[-252:].mean()

    X_tensor = torch.FloatTensor(feat).unsqueeze(0).to(device)
    y_tensor = torch.FloatTensor(rets).unsqueeze(0).to(device)

    train_features_list.append(X_tensor)
    train_targets_list.append(y_tensor)
    all_X = torch.cat(train_features_list, dim=0)
    all_y = torch.cat(train_targets_list, dim=0)

    # Hyperparameter tuning
    print(f"\n=== Tuning for {train_end.date()} ===")
    sampler = optuna.samplers.TPESampler(seed=7)
    study = optuna.create_study(direction="minimize", sampler=sampler)
    study.optimize(
        lambda trial: objective(trial, model, all_X, all_y, rfr),
        n_trials=100,
        timeout=1800,  # 30 minutes max per tuning session
        n_jobs=1,
        gc_after_trial=True
    )

    # Store optimal params
    optimal_params['reg_lambda'].append(study.best_params['reg_lambda'])
    optimal_params['div_lambda'].append(study.best_params['div_lambda'])
    optimal_params['lr'].append(study.best_params['lr'])

    # Full training with best params
    print(f"Training with params: {study.best_params}")
    model = train_model(
        model, all_X, all_y,
        num_epochs=200,
        lr=study.best_params['lr'],
        reg_lambda=study.best_params['reg_lambda'],
        div_lambda=study.best_params['div_lambda'],
        rfr=rfr
    )

    # Portfolio allocation
    with torch.no_grad():
        weights = model(X_tensor).squeeze().cpu().numpy()
        allocation_weights.append(weights)
        backtest_dates.append(test_start)

        test_returns = wide_df.loc[test_start:test_end][[f'close_{tic}' for tic in tics]].pct_change().fillna(0)
        port_ret = (test_returns.values @ weights).sum()
        portfolio_returns.append(port_ret)

# -------------------------
# Calculate Average Optimal Parameters
# -------------------------
avg_params = {
    'avg_reg_lambda': np.mean(optimal_params['reg_lambda']),
    'avg_div_lambda': np.mean(optimal_params['div_lambda']),
    'avg_lr': np.mean(optimal_params['lr'])
}

print("\n=== Average Optimal Parameters ===")
print(f"Regularization Lambda: {avg_params['avg_reg_lambda']:.6f}")
print(f"Diversification Lambda: {avg_params['avg_div_lambda']:.6f}")
print(f"Learning Rate: {avg_params['avg_lr']:.6f}")

# -------------------------
# Backtest Results
# -------------------------
backtest_results = pd.DataFrame({
    'Date': backtest_dates,
    'Weights': allocation_weights,
    'Return': portfolio_returns
})

print("\nBacktest completed!")

cuda


[I 2025-05-24 17:40:55,650] A new study created in memory with name: no-name-dd2d8e38-6fae-42d3-adf6-efdb75ae1578



=== Tuning for 2015-12-31 ===


[I 2025-05-24 17:41:01,114] Trial 0 finished with value: -0.057910602539777756 and parameters: {'reg_lambda': 2.019450244810227e-05, 'div_lambda': 0.013172711156111799, 'lr': 0.00020664632886986634}. Best is trial 0 with value: -0.057910602539777756.
[I 2025-05-24 17:41:01,327] Trial 1 finished with value: -0.2932144105434418 and parameters: {'reg_lambda': 0.007831784183433456, 'div_lambda': 0.08165034948781692, 'lr': 0.00041256046816885543}. Best is trial 1 with value: -0.2932144105434418.
[I 2025-05-24 17:41:01,542] Trial 2 finished with value: 0.021417556330561638 and parameters: {'reg_lambda': 0.001010373285001627, 'div_lambda': 1.9418001640357832e-05, 'lr': 6.387294494138671e-05}. Best is trial 1 with value: -0.2932144105434418.
[I 2025-05-24 17:41:01,749] Trial 3 finished with value: -0.0837254449725151 and parameters: {'reg_lambda': 0.0009989183779852138, 'div_lambda': 0.005210986937158464, 'lr': 0.0025776094086318252}. Best is trial 1 with value: -0.2932144105434418.
[I 2025-05

Training with params: {'reg_lambda': 7.890841691664666e-05, 'div_lambda': 0.09999222335655504, 'lr': 0.003667213073659141}
Epoch 50: Avg Loss = 0.3699
Epoch 100: Avg Loss = 0.3708
Epoch 150: Avg Loss = 0.3707


...

Training with params: {'reg_lambda': 0.00037439493312826504, 'div_lambda': 0.099364353125051, 'lr': 0.0011897638792114668}
Epoch 50: Avg Loss = 0.4519
Epoch 100: Avg Loss = 0.4179
Epoch 150: Avg Loss = 0.4359


[I 2025-05-24 17:53:32,001] A new study created in memory with name: no-name-3cf4ddfd-5ce0-495e-bc33-5203d0d809e3


Epoch 200: Avg Loss = 0.4159

=== Tuning for 2024-09-30 ===


[I 2025-05-24 17:53:32,240] Trial 0 finished with value: -0.17549225687980652 and parameters: {'reg_lambda': 2.019450244810227e-05, 'div_lambda': 0.013172711156111799, 'lr': 0.00020664632886986634}. Best is trial 0 with value: -0.17549225687980652.
[I 2025-05-24 17:53:32,620] Trial 1 finished with value: -0.37370214859644574 and parameters: {'reg_lambda': 0.007831784183433456, 'div_lambda': 0.08165034948781692, 'lr': 0.00041256046816885543}. Best is trial 1 with value: -0.37370214859644574.
[I 2025-05-24 17:53:32,999] Trial 2 finished with value: -0.12894931435585022 and parameters: {'reg_lambda': 0.001010373285001627, 'div_lambda': 1.9418001640357832e-05, 'lr': 6.387294494138671e-05}. Best is trial 1 with value: -0.37370214859644574.
[I 2025-05-24 17:53:33,384] Trial 3 finished with value: -0.15986616412798563 and parameters: {'reg_lambda': 0.0009989183779852138, 'div_lambda': 0.005210986937158464, 'lr': 0.0025776094086318252}. Best is trial 1 with value: -0.37370214859644574.
[I 2025

Training with params: {'reg_lambda': 0.00020847533144764297, 'div_lambda': 0.09917491746255934, 'lr': 0.0011501797296916778}
Epoch 50: Avg Loss = 0.4238
Epoch 100: Avg Loss = 0.4410
Epoch 150: Avg Loss = 0.4315
Epoch 200: Avg Loss = 0.4254

=== Average Optimal Parameters ===
Regularization Lambda: 0.001412
Diversification Lambda: 0.099556
Learning Rate: 0.000788

Backtest completed!




Using random seed of 7 should converge with:


=== Average Optimal Parameters ===

Regularization Lambda: 0.001412

Diversification Lambda: 0.099556

Learning Rate: 0.000788

Backtest completed!

In [7]:
optuna.visualization.plot_optimization_history(study)

In [8]:

optuna.visualization.plot_param_importances(study)